# Requirments

In [ ]:
#First time

'''
from google.colab import drive
drive.mount('/content/drive')

# Navigate to your Google Drive directory and clone the repo there
!git clone https://github.com/icannos/glimpse-mds.git /content/drive/MyDrive/glimpse/glimpse-mds
'''

In [7]:
#Accessing the Repository Later

from google.colab import drive
drive.mount('/content/drive')

# Change to the cloned repository
%cd /content/drive/MyDrive/glimpse/glimpse-mds

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/glimpse/glimpse-mds


In [ ]:
# Install PyTorch with CUDA 12.1 support
!pip install nltk datasets pandas tqdm
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu122
!sudo apt-get install python3.10
!pip install lxml[html_clean]
!pip install sumy transformers rouge_score

In [ ]:
import os
import pandas as pd
import nltk
nltk.download('punkt', force=True)
nltk.download('all')  # To download all resources (use when doing the processing part)
from nltk.tokenize import sent_tokenize
import torch

In [ ]:
!mv requirements requirements.txt
# Install other dependencies
!pip install -r requirements.txt

In [ ]:
# Each file contains columns id, text (review), and gold (metareview)
!python glimpse/data_loading/data_processing.py

# Extractive methods

In [ ]:
# Define the base paths for processed data and output
import os

processed_dir = "/content/drive/MyDrive/glimpse/glimpse-mds/data/processed"
output_dir = "/content/drive/MyDrive/glimpse/glimpse-mds/data/candidates/extractive"

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Define the range of years to process
years = range(2017, 2018)

# Iterate over each year and run the script
for year in years:
    dataset_path = f"{processed_dir}/all_reviews_{year}.csv"
    print(f"Processing year {year}...")

    # Check if input file exists
    if not os.path.exists(dataset_path):
        print(f"Error: Input file {dataset_path} does not exist. Skipping.")
        continue

    # Run the script and check for errors
    exit_code = os.system(f"""
    python /content/drive/MyDrive/glimpse/glimpse-mds/glimpse/data_loading/generate_extractive_candidates.py \
        --dataset_path {dataset_path} \
        --output_dir {output_dir}
    """)
    if exit_code != 0:
        print(f"Error: Script failed for year {year} with exit code {exit_code}")
    else:
        print(f"Year {year} processed successfully!")



Processing year 2017...
Year 2017 processed successfully!


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/glimpse/glimpse-mds')
from rsasumm.rsa_reranker import RSAReranking
print("Import successful")

Import successful


In [ ]:
!CUDA_LAUNCH_BLOCKING=1 python /content/drive/MyDrive/glimpse/glimpse-mds/glimpse/src/compute_rsa.py \
            --model_name facebook/bart-large-cnn \
            --summaries /content/drive/MyDrive/glimpse/glimpse-mds/data/candidates/extractive/extractive_sentences-_-all_reviews_2017-_-none-_-2025-03-24-07-43-18.csv \
            --output_dir /content/drive/MyDrive/glimpse/glimpse-mds/data/results/2017/rsa_scores \
            --device cuda

In [ ]:
!python generate_glimpse_summaries_from_rsa.py \
    --rsa_scores "/content/drive/MyDrive/glimpse/glimpse-mds/data/results/2017/rsa_scores/extractive/extractive_sentences-_-all_reviews_2017-_-none-_-2025-03-24-07-43-18-_-r3-_-rsa_reranked-facebook-bart-large-cnn.pk" \
    --output_dir "/content/drive/MyDrive/glimpse/glimpse-mds/data/results/2017" \
    --year 2017 \
    --n_unique_sentences 3

Loading RSA scores from /content/drive/MyDrive/glimpse/glimpse-mds/data/results/2017/rsa_scores/extractive/extractive_sentences-_-all_reviews_2017-_-none-_-2025-03-24-07-43-18-_-r3-_-rsa_reranked-facebook-bart-large-cnn.pk...

Generating summaries for 490 documents...
100% 490/490 [00:00<00:00, 3576.72it/s]

Summary Statistics:
GLIMPSE-Speaker summaries: 486
GLIMPSE-Unique summaries: 490
Average speaker summary length: 173.70 chars
Average unique summary length: 208.37 chars

Saved summaries to:
- /content/drive/MyDrive/glimpse/glimpse-mds/data/results/2017/speaker_summaries_2017.csv
- /content/drive/MyDrive/glimpse/glimpse-mds/data/results/2017/unique_summaries_2017.csv


In [ ]:
import shutil

# Original file paths
src_path1 = "/content/drive/MyDrive/glimpse/glimpse-mds/data/results/2017/speaker_summaries_2017.csv"
src_path2 = "/content/drive/MyDrive/glimpse/glimpse-mds/data/results/2017/unique_summaries_2017.csv"

# New file paths with new names
dst_path1 = "/content/drive/MyDrive/glimpse/glimpse-mds/data/results/2017/extractive/rough_speaker_summaries_2017.csv"
dst_path2 = "/content/drive/MyDrive/glimpse/glimpse-mds/data/results/2017/extractive/rough_unique_summaries_2017.csv"

# Copy and rename both files
shutil.copy(src_path1, dst_path1)
shutil.copy(src_path2, dst_path2)

# Print confirmation
print(f"File copied to: {dst_path1}")
print(f"File copied to: {dst_path2}")


File copied to: /content/drive/MyDrive/glimpse/glimpse-mds/data/results/2017/extractive/rough_speaker_summaries_2017.csv
File copied to: /content/drive/MyDrive/glimpse/glimpse-mds/data/results/2017/extractive/rough_unique_summaries_2017.csv


In [ ]:
import pandas as pd

# File paths
speaker_path = "/content/drive/MyDrive/glimpse/glimpse-mds/data/results/2017/extractive/rough_speaker_summaries_2017.csv"
unique_path = "/content/drive/MyDrive/glimpse/glimpse-mds/data/results/2017/extractive/rough_unique_summaries_2017.csv"

# Read and modify speaker summaries
speaker_df = pd.read_csv(speaker_path)
speaker_df = speaker_df.rename(columns={'text': 'gold'})
speaker_df.to_csv(speaker_path, index=False)

# Read and modify unique summaries
unique_df = pd.read_csv(unique_path)
unique_df = unique_df.rename(columns={'text': 'gold'})
unique_df.to_csv(unique_path, index=False)

# Verify the changes
print("Speaker summaries columns:", speaker_df.columns.tolist())
print("Unique summaries columns:", unique_df.columns.tolist())

Speaker summaries columns: ['id', 'Method', 'summary', 'gold']
Unique summaries columns: ['id', 'Method', 'summary', 'gold']


In [ ]:
!python /content/drive/MyDrive/glimpse/glimpse-mds/glimpse/evaluate/evaluate_common_metrics_samples.py --summaries /content/drive/MyDrive/glimpse/glimpse-mds/data/results/2017/extractive/rough_speaker_summaries_2017.csv
!python /content/drive/MyDrive/glimpse/glimpse-mds/glimpse/evaluate/evaluate_common_metrics_samples.py --summaries /content/drive/MyDrive/glimpse/glimpse-mds/data/results/2017/extractive/rough_unique_summaries_2017.csv


In [ ]:
import pandas as pd

# Paths to the updated CSVs after running ROUGE evaluation
output_speaker = "/content/drive/MyDrive/glimpse/glimpse-mds/data/results/2017/extractive/rough_speaker_summaries_2017.csv"
output_unique = "/content/drive/MyDrive/glimpse/glimpse-mds/data/results/2017/extractive/rough_unique_summaries_2017.csv"

# Load CSV files
df_speaker = pd.read_csv(output_speaker)
df_unique = pd.read_csv(output_unique)

# Check if ROUGE scores exist
print("Columns in GLIMPSE-Speaker CSV:", df_speaker.columns)
print("Columns in GLIMPSE-Unique CSV:", df_unique.columns)

Columns in GLIMPSE-Speaker CSV: Index(['id', 'Method', 'summary', 'gold', 'common/rouge1', 'common/rouge2',
       'common/rougeL', 'common/rougeLsum'],
      dtype='object')
Columns in GLIMPSE-Unique CSV: Index(['id', 'Method', 'summary', 'gold', 'common/rouge1', 'common/rouge2',
       'common/rougeL', 'common/rougeLsum'],
      dtype='object')


In [ ]:
# Select only ROUGE score columns
rouge_columns = ["common/rouge1", "common/rouge2", "common/rougeL", "common/rougeLsum"]

# Compute mean and std ROUGE scores (as decimals, not percentages)
mean_speaker = df_speaker[rouge_columns].mean()  # Removed * 100
std_speaker = df_speaker[rouge_columns].std()    # Removed * 100
mean_unique = df_unique[rouge_columns].mean()    # Removed * 100
std_unique = df_unique[rouge_columns].std()      # Removed * 100

# Create a formatted table
print("\nROUGE Scores:")
print("=" * 60)
print(f"{'Metric':<12} {'GLIMPSE-Speaker':>22} {'GLIMPSE-Unique':>22}")
print("-" * 60)

metrics_display = {
    "common/rouge1": "ROUGE-1",
    "common/rouge2": "ROUGE-2",
    "common/rougeL": "ROUGE-L",
    "common/rougeLsum": "ROUGE-Lsum"
}

for col in rouge_columns:
    metric_name = metrics_display[col]
    # Format scores to match paper style (0.XX ±0.XX)
    speaker_score = f"{mean_speaker[col]:.2f} ±{std_speaker[col]:.2f}"
    unique_score = f"{mean_unique[col]:.2f} ±{std_unique[col]:.2f}"
    print(f"{metric_name:<12} {speaker_score:>22} {unique_score:>22}")

print("=" * 60)


ROUGE Scores:
Metric              GLIMPSE-Speaker         GLIMPSE-Unique
------------------------------------------------------------
ROUGE-1                  0.17 ±0.07             0.18 ±0.07
ROUGE-2                  0.02 ±0.02             0.03 ±0.03
ROUGE-L                  0.12 ±0.05             0.12 ±0.04
ROUGE-Lsum               0.12 ±0.05             0.12 ±0.04


In [ ]:
import shutil

# Original file paths
src_path1 = "/content/drive/MyDrive/glimpse/glimpse-mds/data/results/2017/speaker_summaries_2017.csv"
src_path2 = "/content/drive/MyDrive/glimpse/glimpse-mds/data/results/2017/unique_summaries_2017.csv"

# New file paths with new names
dst_path1 = "/content/drive/MyDrive/glimpse/glimpse-mds/data/results/2017/extractive/seahorse_speaker_summaries_2017.csv"
dst_path2 = "/content/drive/MyDrive/glimpse/glimpse-mds/data/results/2017/extractive/seahorse_unique_summaries_2017.csv"

# Copy and rename both files
shutil.copy(src_path1, dst_path1)
shutil.copy(src_path2, dst_path2)

# Print confirmation
print(f"File copied to: {dst_path1}")
print(f"File copied to: {dst_path2}")


File copied to: /content/drive/MyDrive/glimpse/glimpse-mds/data/results/2017/extractive/seahorse_speaker_summaries_2017.csv
File copied to: /content/drive/MyDrive/glimpse/glimpse-mds/data/results/2017/extractive/seahorse_unique_summaries_2017.csv


In [ ]:
df_sh = pd.read_csv("/content/drive/MyDrive/glimpse/glimpse-mds/data/results/2017/extractive/seahorse_unique_summaries_2017.csv")
print(df_sh.columns)

Index(['id', 'Method', 'summary', 'text'], dtype='object')


In [ ]:
import pandas as pd

# Path to the CSV file
#file_path = "/content/drive/MyDrive/glimpse/glimpse-mds/data/results/2017/extractive/seahorse_speaker_summaries_2017.csv"
file_path = "/content/drive/MyDrive/glimpse/glimpse-mds/data/results/2017/extractive/seahorse_unique_summaries_2017.csv"

for question in range(1, 7):
    print(f"⚡ Running Seahorse for Question {question}...")

    # Reload dataset before each run to avoid missing columns
    df = pd.read_csv(file_path)

    if "text" not in df.columns or "summary" not in df.columns:
        print("❌ ERROR: 'text' and 'summary' columns missing before running Seahorse. Stopping execution.")
        break  # Stop if important columns are missing

    # Run the Seahorse evaluation script
    !python /content/drive/MyDrive/glimpse/glimpse-mds/glimpse/evaluate/evaluate_seahorse_metrics_samples.py \
        --summaries {file_path} \
        --question {question} \
        --batch_size 8 \
        --device cuda

    # ✅ Verify columns after running Seahorse
    df = pd.read_csv(file_path)
    print(f"✅ After Question {question}, Columns:", df.columns.tolist())

    if "text" not in df.columns or "summary" not in df.columns:
        print("❌ ERROR: 'text' and 'summary' columns missing after running Seahorse. Stopping execution.")
        break


In [ ]:
import pandas as pd

# Read the speaker summaries file
#df_seahorse = pd.read_csv("/content/drive/MyDrive/glimpse/glimpse-mds/data/results/2017/extractive/seahorse_speaker_summaries_2017.csv")
df_seahorse = pd.read_csv("/content/drive/MyDrive/glimpse/glimpse-mds/data/results/2017/extractive/seahorse_unique_summaries_2017.csv")

# Get SEAHORSE metrics (only the proba_1 columns)
seahorse_cols = [col for col in df_seahorse.columns if col.startswith("SHMetric") and col.endswith("proba_1")]

print("\nSEAHORSE Metrics for GLIMPSE:")
print("-" * 50)

if seahorse_cols:
    for col in seahorse_cols:
        metric_name = col.split("/")[1]
        mean_score = df_seahorse[col].mean()
        std_score = df_seahorse[col].std()
        print(f"{metric_name}: {mean_score:.2f} ± {std_score:.2f}")
else:
    print("No SEAHORSE metrics found in the file.")

print("-" * 50)

# Print some basic statistics
print("\nBasic Statistics:")
print(f"Number of summaries: {len(df_seahorse)}")
print(f"Average summary length: {df_seahorse['summary'].str.len().mean():.2f} characters")

## Baseline- extractive methods

In [ ]:
# For LSA
!python /content/drive/MyDrive/glimpse/glimpse-mds/glimpse/baselines/sumy_baselines.py \
  --dataset /content/drive/MyDrive/glimpse/glimpse-mds/data/processed/all_reviews_2017.csv \
  --method LSA \
  --output /content/drive/MyDrive/glimpse/glimpse-mds/data/baseline_summaries/LSA

# For LexRank
!python /content/drive/MyDrive/glimpse/glimpse-mds/glimpse/baselines/sumy_baselines.py \
  --dataset /content/drive/MyDrive/glimpse/glimpse-mds/data/processed/all_reviews_2017.csv \
  --method lex-rank \
  --output /content/drive/MyDrive/glimpse/glimpse-mds/data/baseline_summaries/LexRank


# For Random
!python /content/drive/MyDrive/glimpse/glimpse-mds/glimpse/baselines/sumy_baselines.py \
  --dataset /content/drive/MyDrive/glimpse/glimpse-mds/data/processed/all_reviews_2017.csv \
  --method random \
  --output /content/drive/MyDrive/glimpse/glimpse-mds/data/baseline_summaries/Random

✅ Sample summaries generated:
                                                text                                            summary
0  Summary: The paper presents low-rank bilinear ...  In the caption for Table 1, fix the following:...
1  Results on the VQA task are good for this simp...  Results on the VQA task are good for this simp...
2  This work proposes to approximate the bilinear...  Although the evaluated model is similar to Fuk...
3  Summary:--------This paper proposes to use sur...  This claim is wrong.--------The model requires...
4  This paper proposes to use previous error sign...  This paper proposes to use previous error sign...
✅ Summaries saved at: /content/drive/MyDrive/glimpse/glimpse-mds/data/baseline_summaries/LSA/all_reviews_2017-_-LSA-_-sumy_1.csv
✅ Sample summaries generated:
                                                text                                            summary
0  Summary: The paper presents low-rank bilinear ...  Hence, it could not be experi

In [ ]:
input_rough_file = "/content/drive/MyDrive/glimpse/glimpse-mds/data/baseline_summaries/Random/all_reviews_2017-_-random-_-sumy_1.csv"

df_rough = pd.read_csv(input_rough_file)
# Check if ROUGE scores exist
print("Columns in df_rough CSV:", df_rough.columns)

Columns in df_rough CSV: Index(['id', 'text', 'gold', 'summary', 'metadata/dataset', 'metadata/method',
       'metadata/sentence_count'],
      dtype='object')


In [ ]:

# rough scores for LSA method
#!python /content/drive/MyDrive/glimpse/glimpse-mds/glimpse/evaluate/evaluate_common_metrics_samples.py --summaries /content/drive/MyDrive/glimpse/glimpse-mds/data/baseline_summaries/LSA/rough_all_reviews_2017-_-LSA-_-sumy_1.csv

# rough scores for lex_Rank method
#!python /content/drive/MyDrive/glimpse/glimpse-mds/glimpse/evaluate/evaluate_common_metrics_samples.py --summaries /content/drive/MyDrive/glimpse/glimpse-mds/data/baseline_summaries/LexRank/rough_all_reviews_2017-_-lex-rank-_-sumy_1.csv

# rough scores for Random method
!python /content/drive/MyDrive/glimpse/glimpse-mds/glimpse/evaluate/evaluate_common_metrics_samples.py --summaries /content/drive/MyDrive/glimpse/glimpse-mds/data/baseline_summaries/Random/rough_all_reviews_2017-_-random-_-sumy_1.csv

In [ ]:
#rough_file = "/content/drive/MyDrive/glimpse/glimpse-mds/data/baseline_summaries/LSA/rough_all_reviews_2017-_-LSA-_-sumy_1.csv"
rough_file = "/content/drive/MyDrive/glimpse/glimpse-mds/data/baseline_summaries/LexRank/rough_all_reviews_2017-_-lex-rank-_-sumy_1.csv"
#rough_file = "/content/drive/MyDrive/glimpse/glimpse-mds/data/baseline_summaries/Random/rough_all_reviews_2017-_-random-_-sumy_1.csv"

df_rough = pd.read_csv(rough_file)
# Check if ROUGE scores exist
print("Columns in df_rough CSV:", df_rough.columns)

Columns in df_rough CSV: Index(['id', 'text', 'gold', 'summary', 'metadata/dataset', 'metadata/method',
       'metadata/sentence_count', 'common/rouge1', 'common/rouge2',
       'common/rougeL', 'common/rougeLsum'],
      dtype='object')


In [ ]:
# Select only ROUGE score columns
rouge_columns = ["common/rouge1", "common/rouge2", "common/rougeL", "common/rougeLsum"]

# Compute mean and std ROUGE scores (as decimals, not percentages)
mean_scores = df_rough[rouge_columns].mean()  # Remove the * 100
std_scores = df_rough[rouge_columns].std()    # Remove the * 100

# Create a formatted table
method_name = df_rough['metadata/method'].iloc[0]
print(f"\nROUGE Scores for {method_name}:")
print("=" * 40)
print(f"{'Metric':<12} {'Score':>22}")
print("-" * 40)

metrics_display = {
    "common/rouge1": "ROUGE-1",
    "common/rouge2": "ROUGE-2",
    "common/rougeL": "ROUGE-L",
    "common/rougeLsum": "ROUGE-Lsum"
}

for col in rouge_columns:
    metric_name = metrics_display[col]
    # Format scores to match paper style (0.XX ±0.XX)
    score = f"{mean_scores[col]:.2f} ±{std_scores[col]:.2f}"
    print(f"{metric_name:<12} {score:>22}")

print("=" * 40)

# Print additional statistics
print(f"\nBasic Statistics:")
print(f"Number of summaries: {len(df_rough)}")
print(f"Average summary length: {df_rough['summary'].str.len().mean():.0f} characters")


ROUGE Scores for lex-rank:
Metric                        Score
----------------------------------------
ROUGE-1                  0.17 ±0.09
ROUGE-2                  0.02 ±0.03
ROUGE-L                  0.11 ±0.05
ROUGE-Lsum               0.11 ±0.05

Basic Statistics:
Number of summaries: 1511
Average summary length: 227 characters


In [ ]:
raw_df = pd.read_csv("/content/drive/MyDrive/glimpse/glimpse-mds/data/baseline_summaries/Random/seahorse_all_reviews_2017-_-random-_-sumy_1.csv")

raw_df.head()
print(raw_df.columns)

Index(['id', 'method', 'summary', 'text', 'SHMetric/Comprehensible/proba_1',
       'SHMetric/Comprehensible/proba_0', 'SHMetric/Comprehensible/guess',
       'SHMetric/Repetition/proba_1', 'SHMetric/Repetition/proba_0',
       'SHMetric/Repetition/guess', 'SHMetric/Grammar/proba_1',
       'SHMetric/Grammar/proba_0', 'SHMetric/Grammar/guess',
       'SHMetric/Attribution/proba_1', 'SHMetric/Attribution/proba_0',
       'SHMetric/Attribution/guess', 'SHMetric/Main ideas/proba_1',
       'SHMetric/Main ideas/proba_0', 'SHMetric/Main ideas/guess',
       'SHMetric/Conciseness/proba_1', 'SHMetric/Conciseness/proba_0',
       'SHMetric/Conciseness/guess'],
      dtype='object')


In [ ]:
import pandas as pd
import os

# Path to the CSV file
file_path = "/content/drive/MyDrive/glimpse/glimpse-mds/data/baseline_summaries/Random/seahorse_all_reviews_2017-_-random-_-sumy_1.csv"

# Read the original file and keep only required columns
df = pd.read_csv(file_path)
df = df[['id', 'metadata/method', 'summary', 'text']]
df = df.rename(columns={'metadata/method': 'method'})

# Print sample of the data to verify
print("Sample of input data:")
print(df.head())
print("\nData types:")
print(df.dtypes)
print("\nNumber of rows:", len(df))

In [ ]:
import pandas as pd
import os
import subprocess

# Path to the CSV file

# LSA method
#file_path = "/content/drive/MyDrive/glimpse/glimpse-mds/data/baseline_summaries/LSA/seahorse_all_reviews_2017-_-LSA-_-sumy_1.csv"

#lex_rank method
file_path = "/content/drive/MyDrive/glimpse/glimpse-mds/data/baseline_summaries/LexRank/seahorse_all_reviews_2017-_-lex-rank-_-sumy_1.csv"

#Random method
#file_path = "/content/drive/MyDrive/glimpse/glimpse-mds/data/baseline_summaries/Random/seahorse_all_reviews_2017-_-random-_-sumy_1.csv"

# Read the original file and keep only required columns
df = pd.read_csv(file_path)
df = df[['id', 'metadata/method', 'summary', 'text']]
df = df.rename(columns={'metadata/method': 'method'})

# Create a temporary file in the same directory
temp_file = os.path.join(os.path.dirname(file_path), 'temp_seahorse_eval.csv')
df.to_csv(temp_file, index=False)

# Now run SEAHORSE evaluation on the temporary file
for question in range(1, 7):
    print(f"\n⚡ Running Seahorse for Question {question}...")

    # Run the Seahorse evaluation script on temp file
    cmd = f"python /content/drive/MyDrive/glimpse/glimpse-mds/glimpse/evaluate/evaluate_seahorse_metrics_samples.py --summaries {temp_file} --question {question} --batch_size 8 --device cuda"

    try:
        # Run command and capture output
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True)

        # Print command output
        print(f"Command output:\n{result.stdout}")

        if result.stderr:
            print(f"Errors:\n{result.stderr}")

        # Check if command was successful
        if result.returncode != 0:
            print(f"Command failed with return code {result.returncode}")
            continue

        # Load and check the results after each question
        temp_df = pd.read_csv(temp_file)
        seahorse_cols = [col for col in temp_df.columns if 'SHMetric' in col]

        if seahorse_cols:
            print(f"✅ After Question {question}, Found SEAHORSE columns:")
            for col in seahorse_cols:
                non_nan = temp_df[col].notna().sum()
                print(f"  - {col}: {non_nan} non-NaN values")
        else:
            print(f"❌ No SEAHORSE columns found after Question {question}")

    except Exception as e:
        print(f"Error running question {question}: {str(e)}")
        continue

# After all questions are done, merge results back to original file
try:
    final_df = pd.read_csv(temp_file)

    # Get all SEAHORSE metric columns
    seahorse_columns = [col for col in final_df.columns if 'SHMetric' in col]

    if not seahorse_columns:
        print("❌ No SEAHORSE metrics were calculated!")
    else:
        print(f"\nFound {len(seahorse_columns)} SEAHORSE metric columns")

        # Add SEAHORSE metrics to original DataFrame
        for col in seahorse_columns:
            df[col] = pd.to_numeric(final_df[col], errors='coerce')

        # Print summary of metrics
        print("\nSummary of SEAHORSE metrics:")
        for col in seahorse_columns:
            non_nan = df[col].notna().sum()
            print(f"{col}: {non_nan} non-NaN values")

        # Save the updated original file
        df.to_csv(file_path, index=False)
        print(f"\n✅ Saved results to {file_path}")

except Exception as e:
    print(f"Error processing final results: {str(e)}")

# Clean up temporary file
try:
    os.remove(temp_file)
    print("✅ Cleaned up temporary file")
except Exception as e:
    print(f"Error removing temporary file: {str(e)}")

print("\nFinal columns:", df.columns.tolist())

In [ ]:
#seahorse_scores = pd.read_csv("/content/drive/MyDrive/glimpse/glimpse-mds/data/baseline_summaries/LSA/seahorse_all_reviews_2017-_-LSA-_-sumy_1.csv")
seahorse_scores = pd.read_csv("/content/drive/MyDrive/glimpse/glimpse-mds/data/baseline_summaries/LexRank/seahorse_all_reviews_2017-_-lex-rank-_-sumy_1.csv")
#seahorse_scores = pd.read_csv("/content/drive/MyDrive/glimpse/glimpse-mds/data/baseline_summaries/Random/seahorse_all_reviews_2017-_-random-_-sumy_1.csv")

seahorse_scores.head()
print(seahorse_scores.columns)

Index(['id', 'method', 'summary', 'text', 'SHMetric/Comprehensible/proba_1',
       'SHMetric/Comprehensible/proba_0', 'SHMetric/Comprehensible/guess',
       'SHMetric/Repetition/proba_1', 'SHMetric/Repetition/proba_0',
       'SHMetric/Repetition/guess', 'SHMetric/Grammar/proba_1',
       'SHMetric/Grammar/proba_0', 'SHMetric/Grammar/guess',
       'SHMetric/Attribution/proba_1', 'SHMetric/Attribution/proba_0',
       'SHMetric/Attribution/guess', 'SHMetric/Main ideas/proba_1',
       'SHMetric/Main ideas/proba_0', 'SHMetric/Main ideas/guess',
       'SHMetric/Conciseness/proba_1', 'SHMetric/Conciseness/proba_0',
       'SHMetric/Conciseness/guess'],
      dtype='object')


In [ ]:
import pandas as pd

# Load the Seahorse scores CSV file

# LSA method
#file_path = "/content/drive/MyDrive/glimpse/glimpse-mds/data/baseline_summaries/LSA/seahorse_all_reviews_2017-_-LSA-_-sumy_1.csv"

#lex_rank method
file_path = "/content/drive/MyDrive/glimpse/glimpse-mds/data/baseline_summaries/LexRank/seahorse_all_reviews_2017-_-lex-rank-_-sumy_1.csv"

#Random method
#file_path = "/content/drive/MyDrive/glimpse/glimpse-mds/data/baseline_summaries/Random/seahorse_all_reviews_2017-_-random-_-sumy_1.csv"


seahorse_scores = pd.read_csv(file_path)

# Select the relevant columns for calculation, including 'Main Ideas' as 'Coverage'
columns_to_average = {
    "Coverage": "SHMetric/Main ideas/proba_1",  # ✅ Added
    "Comprehensible": "SHMetric/Comprehensible/proba_1",
    "Attribution": "SHMetric/Attribution/proba_1",
    "Grammar": "SHMetric/Grammar/proba_1",
    "Conciseness": "SHMetric/Conciseness/proba_1",
    "Repetition": "SHMetric/Repetition/proba_1"
}

# Compute mean ± std deviation for each metric
summary_results = {}
for metric, column in columns_to_average.items():
    if column in seahorse_scores.columns:
        mean_value = seahorse_scores[column].mean()
        std_value = seahorse_scores[column].std()
        summary_results[metric] = f"{mean_value:.2f} ± {std_value:.2f}"

# Print the computed results
summary_results



{'Coverage': '0.26 ± 0.23',
 'Comprehensible': '0.93 ± 0.10',
 'Attribution': '0.93 ± 0.06',
 'Grammar': '0.77 ± 0.20',
 'Conciseness': '0.38 ± 0.17',
 'Repetition': '0.97 ± 0.09'}

# Abstractive methods

In [ ]:
# Define the base paths for processed data and output
import os

processed_dir = "/content/drive/MyDrive/glimpse/glimpse-mds/data/processed"
output_dir = "/content/drive/MyDrive/glimpse/glimpse-mds/data/candidates/abstractive"
model_name = "facebook/bart-large-cnn"
decoding_config = "beam_search"  # ✅ Changed to Beam Search
batch_size = 8

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Define the range of years to process
years = range(2017, 2018)

# Loop through each year and run the script
for year in years:
    dataset_path = f"{processed_dir}/all_reviews_{year}.csv"
    !python /content/drive/MyDrive/glimpse/glimpse-mds/glimpse/data_loading/generate_abstractive_candidates.py \
        --model_name {model_name} \
        --dataset_path {dataset_path} \
        --output_dir {output_dir} \
        --batch_size {batch_size} \
        --decoding_config {decoding_config} \
        --no-trimming

print("All years processed successfully!")

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/glimpse/glimpse-mds')
from rsasumm.rsa_reranker import RSAReranking
print("Import successful")

Import successful


In [ ]:
!CUDA_LAUNCH_BLOCKING=1 python /content/drive/MyDrive/glimpse/glimpse-mds/glimpse/src/compute_rsa.py \
            --model_name facebook/bart-large-cnn \
            --summaries /content/drive/MyDrive/glimpse/glimpse-mds/data/candidates/abstractive/facebook_bart-large-cnn-_-all_reviews_2017-_-beam_search-_-trimmed-_-2025-03-26-10-31-47.csv \
            --output_dir /content/drive/MyDrive/glimpse/glimpse-mds/data/results/2017/rsa_scores/abstractive \
            --device cuda

In [ ]:
!python /content/drive/MyDrive/glimpse/glimpse-mds/generate_glimpse_summaries_from_rsa.py \
    --rsa_scores "/content/drive/MyDrive/glimpse/glimpse-mds/data/results/2017/rsa_scores/abstractive/facebook_bart-large-cnn-_-all_reviews_2017-_-beam_search-_-trimmed-_-2025-03-26-10-31-47-_-r3-_-rsa_reranked-facebook-bart-large-cnn.pk" \
    --output_dir "/content/drive/MyDrive/glimpse/glimpse-mds/data/results/2017/abstractive" \
    --year 2017 \
    --n_unique_sentences 3

Loading RSA scores from /content/drive/MyDrive/glimpse/glimpse-mds/data/results/2017/rsa_scores/abstractive/facebook_bart-large-cnn-_-all_reviews_2017-_-beam_search-_-trimmed-_-2025-03-26-10-31-47-_-r3-_-rsa_reranked-facebook-bart-large-cnn.pk...

Generating summaries for 490 documents...
100% 490/490 [00:00<00:00, 3965.45it/s]

Summary Statistics:
GLIMPSE-Speaker summaries: 488
GLIMPSE-Unique summaries: 490
Average speaker summary length: 229.41 chars
Average unique summary length: 227.02 chars

Saved summaries to:
- /content/drive/MyDrive/glimpse/glimpse-mds/data/results/2017/abstractive/speaker_summaries_2017.csv
- /content/drive/MyDrive/glimpse/glimpse-mds/data/results/2017/abstractive/unique_summaries_2017.csv


In [ ]:
import pandas as pd

# Read the generated files
speaker_df = pd.read_csv("/content/drive/MyDrive/glimpse/glimpse-mds/data/results/2017/abstractive/rough_speaker_summaries_2017.csv")
unique_df = pd.read_csv("/content/drive/MyDrive/glimpse/glimpse-mds/data/results/2017/abstractive/rough_unique_summaries_2017.csv")

print("Speaker summaries columns:", speaker_df.columns.tolist())
print("Unique summaries columns:", unique_df.columns.tolist())

Speaker summaries columns: ['id', 'Method', 'summary', 'gold', 'common/rouge1', 'common/rouge2', 'common/rougeL', 'common/rougeLsum']
Unique summaries columns: ['id', 'Method', 'summary', 'gold', 'common/rouge1', 'common/rouge2', 'common/rougeL', 'common/rougeLsum']


In [ ]:
import pandas as pd

# File paths
speaker_path = "/content/drive/MyDrive/glimpse/glimpse-mds/data/results/2017/abstractive/rough_speaker_summaries_2017.csv"
unique_path = "/content/drive/MyDrive/glimpse/glimpse-mds/data/results/2017/abstractive/rough_unique_summaries_2017.csv"

# Read and modify speaker summaries
speaker_df = pd.read_csv(speaker_path)
speaker_df = speaker_df.rename(columns={'text': 'gold'})
speaker_df.to_csv(speaker_path, index=False)

# Read and modify unique summaries
unique_df = pd.read_csv(unique_path)
unique_df = unique_df.rename(columns={'text': 'gold'})
unique_df.to_csv(unique_path, index=False)

# Verify the changes
print("Speaker summaries columns:", speaker_df.columns.tolist())
print("Unique summaries columns:", unique_df.columns.tolist())

Speaker summaries columns: ['id', 'Method', 'summary', 'gold', 'common/rouge1', 'common/rouge2', 'common/rougeL', 'common/rougeLsum']
Unique summaries columns: ['id', 'Method', 'summary', 'gold', 'common/rouge1', 'common/rouge2', 'common/rougeL', 'common/rougeLsum']


In [ ]:
!python /content/drive/MyDrive/glimpse/glimpse-mds/glimpse/evaluate/evaluate_common_metrics_samples.py --summaries /content/drive/MyDrive/glimpse/glimpse-mds/data/results/2017/abstractive/rough_speaker_summaries_2017.csv
!python /content/drive/MyDrive/glimpse/glimpse-mds/glimpse/evaluate/evaluate_common_metrics_samples.py --summaries /content/drive/MyDrive/glimpse/glimpse-mds/data/results/2017/abstractive/rough_unique_summaries_2017.csv


In [ ]:
import pandas as pd

# Paths to the updated CSVs after running ROUGE evaluation
output_speaker = "/content/drive/MyDrive/glimpse/glimpse-mds/data/results/2017/abstractive/rough_speaker_summaries_2017.csv"
output_unique = "/content/drive/MyDrive/glimpse/glimpse-mds/data/results/2017/abstractive/rough_unique_summaries_2017.csv"

# Load CSV files
df_speaker = pd.read_csv(output_speaker)
df_unique = pd.read_csv(output_unique)

# Check if ROUGE scores exist
print("Columns in GLIMPSE-Speaker CSV:", df_speaker.columns)
print("Columns in GLIMPSE-Unique CSV:", df_unique.columns)

Columns in GLIMPSE-Speaker CSV: Index(['id', 'Method', 'summary', 'gold', 'common/rouge1', 'common/rouge2',
       'common/rougeL', 'common/rougeLsum'],
      dtype='object')
Columns in GLIMPSE-Unique CSV: Index(['id', 'Method', 'summary', 'gold', 'common/rouge1', 'common/rouge2',
       'common/rougeL', 'common/rougeLsum'],
      dtype='object')


In [ ]:
# Select only ROUGE score columns
rouge_columns = ["common/rouge1", "common/rouge2", "common/rougeL", "common/rougeLsum"]

# Compute mean and std ROUGE scores (as decimals, not percentages)
mean_speaker = df_speaker[rouge_columns].mean()  # Removed * 100
std_speaker = df_speaker[rouge_columns].std()    # Removed * 100
mean_unique = df_unique[rouge_columns].mean()    # Removed * 100
std_unique = df_unique[rouge_columns].std()      # Removed * 100

# Create a formatted table
print("\nROUGE Scores:")
print("=" * 60)
print(f"{'Metric':<12} {'GLIMPSE-Speaker':>22} {'GLIMPSE-Unique':>22}")
print("-" * 60)

metrics_display = {
    "common/rouge1": "ROUGE-1",
    "common/rouge2": "ROUGE-2",
    "common/rougeL": "ROUGE-L",
    "common/rougeLsum": "ROUGE-Lsum"
}

for col in rouge_columns:
    metric_name = metrics_display[col]
    # Format scores to match paper style (0.XX ±0.XX)
    speaker_score = f"{mean_speaker[col]:.2f} ±{std_speaker[col]:.2f}"
    unique_score = f"{mean_unique[col]:.2f} ±{std_unique[col]:.2f}"
    print(f"{metric_name:<12} {speaker_score:>22} {unique_score:>22}")

print("=" * 60)


ROUGE Scores:
Metric              GLIMPSE-Speaker         GLIMPSE-Unique
------------------------------------------------------------
ROUGE-1                  0.19 ±0.07             0.18 ±0.07
ROUGE-2                  0.03 ±0.02             0.03 ±0.02
ROUGE-L                  0.13 ±0.04             0.12 ±0.04
ROUGE-Lsum               0.13 ±0.04             0.12 ±0.04


In [ ]:
df_sh = pd.read_csv("/content/drive/MyDrive/glimpse/glimpse-mds/data/results/2017/abstractive/seahorse_unique_summaries_2017.csv")
print(df_sh.columns)

In [ ]:
import pandas as pd

# Path to the CSV file
#file_path = "/content/drive/MyDrive/glimpse/glimpse-mds/data/results/2017/abstractive/seahorse_speaker_summaries_2017.csv"
file_path = "/content/drive/MyDrive/glimpse/glimpse-mds/data/results/2017/abstractive/seahorse_unique_summaries_2017.csv"

for question in range(1, 7):
    print(f"⚡ Running Seahorse for Question {question}...")

    # Reload dataset before each run to avoid missing columns
    df = pd.read_csv(file_path)

    if "text" not in df.columns or "summary" not in df.columns:
        print("❌ ERROR: 'text' and 'summary' columns missing before running Seahorse. Stopping execution.")
        break  # Stop if important columns are missing

    # Run the Seahorse evaluation script
    !python /content/drive/MyDrive/glimpse/glimpse-mds/glimpse/evaluate/evaluate_seahorse_metrics_samples.py \
        --summaries {file_path} \
        --question {question} \
        --batch_size 8 \
        --device cuda

    # ✅ Verify columns after running Seahorse
    df = pd.read_csv(file_path)
    print(f"✅ After Question {question}, Columns:", df.columns.tolist())

    if "text" not in df.columns or "summary" not in df.columns:
        print("❌ ERROR: 'text' and 'summary' columns missing after running Seahorse. Stopping execution.")
        break


⚡ Running Seahorse for Question 1...
2025-06-12 16:53:19.013425: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749747199.051540   26342 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749747199.063908   26342 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-12 16:53:19.105802: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Traceback (most recent call last):
  File "/content/drive/MyDrive/glimpse/glimpse-mds/gl

In [ ]:
import pandas as pd

# Read the speaker summaries file
#df_seahorse = pd.read_csv("/content/drive/MyDrive/glimpse/glimpse-mds/data/results/2017/abstractive/seahorse_speaker_summaries_2017.csv")
df_seahorse = pd.read_csv("/content/drive/MyDrive/glimpse/glimpse-mds/data/results/2017/abstractive/seahorse_unique_summaries_2017.csv")

# Get SEAHORSE metrics (only the proba_1 columns)
seahorse_cols = [col for col in df_seahorse.columns if col.startswith("SHMetric") and col.endswith("proba_1")]

print("\nSEAHORSE Metrics for GLIMPSE:")
print("-" * 50)

if seahorse_cols:
    for col in seahorse_cols:
        metric_name = col.split("/")[1]
        mean_score = df_seahorse[col].mean()
        std_score = df_seahorse[col].std()
        print(f"{metric_name}: {mean_score:.2f} ± {std_score:.2f}")
else:
    print("No SEAHORSE metrics found in the file.")

print("-" * 50)

# Print some basic statistics
print("\nBasic Statistics:")
print(f"Number of summaries: {len(df_seahorse)}")
print(f"Average summary length: {df_seahorse['summary'].str.len().mean():.2f} characters")


SEAHORSE Metrics for GLIMPSE:
--------------------------------------------------
Comprehensible: 0.92 ± 0.10
Repetition: 0.97 ± 0.04
Grammar: 0.77 ± 0.21
Attribution: 0.08 ± 0.08
Main ideas: 0.05 ± 0.10
Conciseness: 0.06 ± 0.06
--------------------------------------------------

Basic Statistics:
Number of summaries: 490
Average summary length: 227.02 characters


# Extentions

## Extention1:PubMed Adaptation

### 1.Load the data from CSV files:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [8]:
import pandas as pd

# Load PubMed dataset (assuming CSV format)
train_df = pd.read_csv("/content/drive/MyDrive/glimpse/pubmed/pubmed_train.csv")
valid_df = pd.read_csv("/content/drive/MyDrive/glimpse/pubmed/pubmed_val.csv")
test_df = pd.read_csv("/content/drive/MyDrive/glimpse/pubmed/pubmed_test.csv")

# Check dataset structure
print(train_df.columns)

Index(['article', 'abstract', 'section_names'], dtype='object')


In [9]:
# Drop missing values
train_df = train_df.dropna(subset=['article', 'abstract'])
valid_df = valid_df.dropna(subset=['article', 'abstract'])
test_df = test_df.dropna(subset=['article', 'abstract'])

In [10]:
# Rename necessary columns
train_df.rename(columns={"article": "text", "abstract": "summary"}, inplace=True)
valid_df.rename(columns={"article": "text", "abstract": "summary"}, inplace=True)
test_df.rename(columns={"article": "text", "abstract": "summary"}, inplace=True)

# Check dataset structure
print(test_df.columns)

Index(['text', 'summary', 'section_names'], dtype='object')


In [11]:
# Save processed files
train_df.to_csv("/content/drive/MyDrive/glimpse/pubmed/proccessed/processed_train.csv", index=False)
valid_df.to_csv("/content/drive/MyDrive/glimpse/pubmed/proccessed/processed_valid.csv", index=False)
test_df.to_csv("/content/drive/MyDrive/glimpse/pubmed/proccessed/processed_test.csv", index=False)

print("✅ PubMed dataset formatted successfully!")

✅ PubMed dataset formatted successfully!


In [12]:
# Count missing values per column
print(train_df.isna().sum())
print(valid_df.isna().sum())
print(test_df.isna().sum())

text              0
summary           0
section_names    18
dtype: int64
text             0
summary          0
section_names    1
dtype: int64
text             0
summary          0
section_names    2
dtype: int64


In [13]:
print(test_df.shape)

(6658, 3)


## 2.Fine-Tune BART on PubMed

In [ ]:
!pip install deepspeed --quiet
!pip install --upgrade transformers accelerate --no-cache-dir
!pip install --upgrade pip setuptools wheel
!pip install -U bitsandbytes
!pip uninstall bitsandbytes -y
!pip install bitsandbytes
!pip install nltk datasets pandas tqdm

In [ ]:
import os
import torch
import pandas as pd
from transformers import TrainingArguments, Trainer, AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import Dataset

# ✅ Free GPU memory
torch.cuda.empty_cache()

# ✅ Load tokenizer & model
model_name = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


# ✅ Move model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)


# ✅ Load training & validation datasets
train_df = pd.read_csv("/content/drive/MyDrive/glimpse/pubmed/proccessed/processed_train.csv")
valid_df = pd.read_csv("/content/drive/MyDrive/glimpse/pubmed/proccessed/processed_valid.csv")

# ✅ Reduce training dataset size to 25% (if needed for speed)
train_df = train_df.sample(frac=0.25, random_state=42)

# ✅ Convert to Hugging Face datasets
train_dataset = Dataset.from_pandas(train_df)
valid_dataset = Dataset.from_pandas(valid_df)

# ✅ Tokenization function
def tokenize_function(examples):
    model_inputs = tokenizer(
        examples["text"], max_length=1024, truncation=True, padding="max_length"
    )
    labels = tokenizer(
        examples["summary"], max_length=512, truncation=True, padding="max_length"
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# ✅ Apply tokenization
train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=["text", "summary"])
valid_dataset = valid_dataset.map(tokenize_function, batched=True, remove_columns=["text", "summary"])

print("✅ Tokenization completed!")

# ✅ Training Arguments (Lower Learning Rate for Stability)
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/glimpse/pubmed/fine-tuned-bart",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=1,  # ✅ Adjust batch size (Reduce if OOM)
    per_device_eval_batch_size=1,
    num_train_epochs=5,
    save_total_limit=2,  # ✅ Keep only last 2 checkpoints
    learning_rate=1e-5,  # 🔹 Reduce LR to avoid overfitting
    fp16=True,  # ✅ Enable mixed precision for efficiency
    logging_dir="/content/drive/MyDrive/glimpse/pubmed/logs",
    logging_steps=500,
    report_to="none",
)

# ✅ Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
)

# ✅ Train & Save After Each Epoch (Fixed Epoch Numbering)
for epoch in range(1, 6):
    print(f"🚀 Starting Epoch {epoch}/5...")
    trainer.train()

    # ✅ Convert model back to FP32 before saving (Fix FP16 issues)
    model = model.float()

    save_dir = f"/content/drive/MyDrive/glimpse/pubmed/fine-tuned-bart/epoch-{epoch}"
    model.save_pretrained(save_dir)
    tokenizer.save_pretrained(save_dir)
    print(f"✅ Model saved at {save_dir}")

# ✅ Save Final Model in FP32
model.save_pretrained("/content/drive/MyDrive/glimpse/pubmed/fine-tuned-bart")
tokenizer.save_pretrained("/content/drive/MyDrive/glimpse/pubme/fine-tuned-bart-final")

print("✅ Fine-tuning completed and final model saved in FP32!")

## 4. Generate Summaries Using Fine-Tuned Model

In [17]:
import pandas as pd

# Load dataset and check if it's properly loaded
df_test = pd.read_csv("/content/drive/MyDrive/glimpse/pubmed/proccessed/processed_test.csv")
print(df_test.columns)  # Should show: ['text', 'gold', 'section_names']
print(df_test.isnull().sum())  # To check if there are any NaN values
print(df_test.head())  # To see the first few rows
print(df_test.shape)

Index(['text', 'summary', 'section_names'], dtype='object')
text             0
summary          0
section_names    2
dtype: int64
                                                text  \
0  anxiety affects quality of life in those livin...   
1  small non - coding rnas are transcribed into m...   
2  ohss is a serious complication of ovulation in...   
3  congenital adrenal hyperplasia ( cah ) refers ...   
4  type 1 diabetes ( t1d ) results from the destr...   

                                             summary  \
0   research on the implications of anxiety in pa...   
1   small non - coding rnas include sirna , mirna...   
2   objective : to evaluate the efficacy and safe...   
3   congenital adrenal hyperplasia is a group of ...   
4   objective(s):pentoxifylline is an immunomodul...   

                                       section_names  
0  1. Introduction\n2. Methods\n3. Results\n4. Di...  
1  Introduction\nAberrant Expression of miRNA in ...  
2  Introduction\nMaterials and 

A. Abstractive candidates

In [ ]:
import torch
import pandas as pd
from transformers import BartForConditionalGeneration, AutoTokenizer
from tqdm import tqdm
import os
from datetime import datetime

# ✅ Load Fine-Tuned Model & Tokenizer
model_path = "/content/drive/MyDrive/glimpse/pubmed/fine-tuned-bart-final"
model = BartForConditionalGeneration.from_pretrained(model_path).to("cuda")
tokenizer = AutoTokenizer.from_pretrained(model_path)

# ✅ Fix Missing Token Issues (Ensure correct settings)
model.config.forced_bos_token_id = tokenizer.bos_token_id
model.config.decoder_start_token_id = tokenizer.pad_token_id

# ✅ Load Test Dataset
test_df_path = "/content/drive/MyDrive/glimpse/pubmed/proccessed/processed_test.csv"
test_df = pd.read_csv(test_df_path)

# ✅ Remove Null Rows (if any)
test_df.dropna(subset=["text"], inplace=True)

# ✅ Function to Generate Summaries
def generate_summary(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=1024, padding="max_length").to("cuda")

    with torch.no_grad():  # Disable gradient calculation for speedup
        outputs = model.generate(
            **inputs,
            max_new_tokens=200,
            num_beams=4,
            early_stopping=True,
            no_repeat_ngram_size=3,  # Prevent repetition
        )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# ✅ Generate Summaries for the Entire Dataset
summaries = []
for text in tqdm(test_df["text"], desc="Generating Summaries"):
    summaries.append(generate_summary(text))

# ✅ Add Summaries to the DataFrame
test_df["summary"] = summaries

# ✅ Reset index to create 'index' column
test_df = test_df.reset_index()

# ✅ Add id_candidate column (same as index in this case since we're generating one summary per text)
test_df['id_candidate'] = test_df['index']

# ✅ Create output directory and generate filename with timestamp
output_dir = "/content/drive/MyDrive/glimpse/pubmed/abstractive_summaries"
os.makedirs(output_dir, exist_ok=True)
timestamp = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
output_file = os.path.join(output_dir, f"abstractive_summaries_{timestamp}.csv")

# ✅ Save the Results
test_df.to_csv(output_file, index=False)

print(f"✅ Summaries saved to: {output_file}")
print(f"Columns in saved file: {test_df.columns.tolist()}")

In [19]:
import sys
sys.path.append('/content/drive/MyDrive/glimpse/glimpse-mds')
from rsasumm.rsa_reranker import RSAReranking
print("Import successful")

Import successful


In [20]:
file_path = "/content/drive/MyDrive/glimpse/pubmed/abstractive_summaries/abstractive_summaries_2025-04-02-02-13-59.csv"
df= pd.read_csv(file_path)

print(df.columns)

Index(['index', 'text', 'gold', 'section_names', 'summary', 'id_candidate',
       'id'],
      dtype='object')


In [21]:
import pandas as pd

def add_id_column_if_missing(file_path):
    """
    Check if CSV file has 'id' column, if not add it and update the file
    Args:
        file_path: Path to the CSV file
    """
    try:
        # Read the CSV file
        print(f"📄 Reading file: {file_path}")
        df = pd.read_csv(file_path)

        # Print current columns
        print(f"Current columns: {df.columns.tolist()}")

        # Check if 'id' column exists
        if 'id' not in df.columns:
            print("'id' column not found. Adding it...")
            # Add 'id' column (using index + 1 to start from 1)
            df['id'] = df.index + 1

            # Save back to the same file
            df.to_csv(file_path, index=False)
            print(f"✅ Added 'id' column and updated file: {file_path}")
            print(f"New columns: {df.columns.tolist()}")
        else:
            print("'id' column already exists. No changes needed.")

        return df

    except Exception as e:
        print(f"❌ Error: {str(e)}")
        return None

# Example usage:
file_path = "/content/drive/MyDrive/glimpse/pubmed/abstractive_summaries/abstractive_summaries_2025-04-02-02-13-59.csv"
df = add_id_column_if_missing(file_path)

📄 Reading file: /content/drive/MyDrive/glimpse/pubmed/abstractive_summaries/abstractive_summaries_2025-04-02-02-13-59.csv
Current columns: ['index', 'text', 'gold', 'section_names', 'summary', 'id_candidate', 'id']
'id' column already exists. No changes needed.


In [22]:
import pandas as pd

# Read and display the first few rows of your CSV file
file_path = "/content/drive/MyDrive/glimpse/pubmed/abstractive_summaries/abstractive_summaries_2025-04-02-02-13-59.csv"
df = pd.read_csv(file_path)
print("First few rows of the CSV file:")
print(df.head())
print("\nDataFrame info:")
print(df.info())

First few rows of the CSV file:
   index                                               text  \
0      0  anxiety affects quality of life in those livin...   
1      1  small non - coding rnas are transcribed into m...   
2      2  ohss is a serious complication of ovulation in...   
3      3  congenital adrenal hyperplasia ( cah ) refers ...   
4      4  type 1 diabetes ( t1d ) results from the destr...   

                                                gold  \
0   research on the implications of anxiety in pa...   
1   small non - coding rnas include sirna , mirna...   
2   objective : to evaluate the efficacy and safe...   
3   congenital adrenal hyperplasia is a group of ...   
4   objective(s):pentoxifylline is an immunomodul...   

                                       section_names  \
0  1. Introduction\n2. Methods\n3. Results\n4. Di...   
1  Introduction\nAberrant Expression of miRNA in ...   
2  Introduction\nMaterials and Methods\nResults\n...   
3                           

In [ ]:
!CUDA_LAUNCH_BLOCKING=1 python /content/drive/MyDrive/glimpse/glimpse-mds/glimpse/src/compute_rsa.py \
            --model_name facebook/bart-large-cnn \
            --summaries /content/drive/MyDrive/glimpse/pubmed/extractive_summaries/extractive_sentences-_-processed_test-_-none-_-2025-03-27-11-58-15.csv \
            --output_dir /content/drive/MyDrive/glimpse/pubmed/extractive_summaries \
            --device cuda

In [24]:
!python generate_glimpse_summaries_from_rsa.py \
    --rsa_scores "/content/drive/MyDrive/glimpse/pubmed/rsa_scores/abstractive_summaries_2025-04-02-02-13-59-_-r3-_-rsa_reranked-facebook-bart-large-cnn.pk"\
    --output_dir "/content/drive/MyDrive/glimpse/pubmed/abstractive_summaries" \
    --year "pubmed" \
    --n_unique_sentences 3

Loading RSA scores from /content/drive/MyDrive/glimpse/pubmed/rsa_scores/abstractive_summaries_2025-04-02-02-13-59-_-r3-_-rsa_reranked-facebook-bart-large-cnn.pk...

Generating summaries for 6658 documents...
100% 6658/6658 [00:01<00:00, 3367.48it/s]

Summary Statistics:
GLIMPSE-Speaker summaries: 6658
GLIMPSE-Unique summaries: 6658
Average speaker summary length: 221.00 chars
Average unique summary length: 220.27 chars

Saved summaries to:
- /content/drive/MyDrive/glimpse/pubmed/abstractive_summaries/speaker_summaries_pubmed.csv
- /content/drive/MyDrive/glimpse/pubmed/abstractive_summaries/unique_summaries_pubmed.csv


In [25]:
import pandas as pd

# Read the generated files
abstractive_pubmed_df = pd.read_csv("/content/drive/MyDrive/glimpse/pubmed/abstractive_summaries/rough_unique_summaries_pubmed.csv")

print("abstractive pubmed summaries columns:", abstractive_pubmed_df.columns.tolist())

abstractive pubmed summaries columns: ['id', 'Method', 'summary', 'gold', 'common/rouge1', 'common/rouge2', 'common/rougeL', 'common/rougeLsum']


In [ ]:
import pandas as pd

# File paths
speaker_path = "/content/drive/MyDrive/glimpse/pubmed/abstractive_summaries/rough_speaker_summaries_pubmed.csv"
unique_path = "/content/drive/MyDrive/glimpse/pubmed/abstractive_summaries/rough_unique_summaries_pubmed.csv"

# Read and modify speaker summaries
speaker_df = pd.read_csv(speaker_path)
speaker_df = speaker_df.rename(columns={'text': 'gold'})
speaker_df.to_csv(speaker_path, index=False)

# Read and modify unique summaries
unique_df = pd.read_csv(unique_path)
unique_df = unique_df.rename(columns={'text': 'gold'})
unique_df.to_csv(unique_path, index=False)

# Verify the changes
print("Speaker summaries columns:", speaker_df.columns.tolist())
print("Unique summaries columns:", unique_df.columns.tolist())




!python /content/drive/MyDrive/glimpse/glimpse-mds/glimpse/evaluate/evaluate_common_metrics_samples.py --summaries /content/drive/MyDrive/glimpse/pubmed/abstractive_summaries/rough_speaker_summaries_pubmed.csv
!python /content/drive/MyDrive/glimpse/glimpse-mds/glimpse/evaluate/evaluate_common_metrics_samples.py --summaries /content/drive/MyDrive/glimpse/pubmed/abstractive_summaries/rough_unique_summaries_pubmed.csv




import pandas as pd

# Paths to the updated CSVs after running ROUGE evaluation
output_speaker = "/content/drive/MyDrive/glimpse/pubmed/abstractive_summaries/rough_speaker_summaries_pubmed.csv"
output_unique = "/content/drive/MyDrive/glimpse/pubmed/abstractive_summaries/rough_unique_summaries_pubmed.csv"

# Load CSV files
df_speaker = pd.read_csv(output_speaker)
df_unique = pd.read_csv(output_unique)

# Check if ROUGE scores exist
print("Columns in GLIMPSE-Speaker CSV:", df_speaker.columns)
print("Columns in GLIMPSE-Unique CSV:", df_unique.columns)





# Select only ROUGE score columns
rouge_columns = ["common/rouge1", "common/rouge2", "common/rougeL", "common/rougeLsum"]

# Compute mean and std ROUGE scores (as decimals, not percentages)
mean_speaker = df_speaker[rouge_columns].mean()  # Removed * 100
std_speaker = df_speaker[rouge_columns].std()    # Removed * 100
mean_unique = df_unique[rouge_columns].mean()    # Removed * 100
std_unique = df_unique[rouge_columns].std()      # Removed * 100

# Create a formatted table
print("\nROUGE Scores:")
print("=" * 60)
print(f"{'Metric':<12} {'GLIMPSE-Speaker':>22} {'GLIMPSE-Unique':>22}")
print("-" * 60)

metrics_display = {
    "common/rouge1": "ROUGE-1",
    "common/rouge2": "ROUGE-2",
    "common/rougeL": "ROUGE-L",
    "common/rougeLsum": "ROUGE-Lsum"
}

for col in rouge_columns:
    metric_name = metrics_display[col]
    # Format scores to match paper style (0.XX ±0.XX)
    speaker_score = f"{mean_speaker[col]:.2f} ±{std_speaker[col]:.2f}"
    unique_score = f"{mean_unique[col]:.2f} ±{std_unique[col]:.2f}"
    print(f"{metric_name:<12} {speaker_score:>22} {unique_score:>22}")

print("=" * 60)

In [ ]:
import pandas as pd

# Path to the CSV file
#file_path = "/content/drive/MyDrive/glimpse/pubmed/abstractive_summaries/seahorse_speaker_summaries_pubmed.csv"
file_path = "/content/drive/MyDrive/glimpse/pubmed/abstractive_summaries/seahorse_unique_summaries_pubmed.csv"

for question in range(1, 7):
    print(f"⚡ Running Seahorse for Question {question}...")

    # Reload dataset before each run to avoid missing columns
    df = pd.read_csv(file_path)

    if "text" not in df.columns or "summary" not in df.columns:
        print("❌ ERROR: 'text' and 'summary' columns missing before running Seahorse. Stopping execution.")
        break  # Stop if important columns are missing

    # Run the Seahorse evaluation script
    !python /content/drive/MyDrive/glimpse/glimpse-mds/glimpse/evaluate/Copyof_evaluate_seahorse_metrics_samples.py \
        --summaries {file_path} \
        --question {question} \
        --batch_size 8 \
        --device cuda


    # ✅ Verify columns after running Seahorse
    df = pd.read_csv(file_path)
    print(f"✅ After Question {question}, Columns:", df.columns.tolist())

    if "text" not in df.columns or "summary" not in df.columns:
        print("❌ ERROR: 'text' and 'summary' columns missing after running Seahorse. Stopping execution.")
        break


In [3]:
import pandas as pd

# Path to the CSV file
#file_path = "/content/drive/MyDrive/glimpse/pubmed/abstractive_summaries/seahorse_speaker_summaries_pubmed.csv"
file_path = "/content/drive/MyDrive/glimpse/pubmed/abstractive_summaries/seahorse_unique_summaries_pubmed.csv"

# Read the CSV file
df_seahorse = pd.read_csv(file_path)

# Get SEAHORSE metrics (only the proba_1 columns)
seahorse_cols = [col for col in df_seahorse.columns if col.startswith("SHMetric") and col.endswith("proba_1")]

print("\nSEAHORSE Metrics for GLIMPSE:")
print("-" * 50)

if seahorse_cols:
    for col in seahorse_cols:
        metric_name = col.split("/")[1]
        mean_score = df_seahorse[col].mean()
        std_score = df_seahorse[col].std()
        print(f"{metric_name}: {mean_score:.2f} ± {std_score:.2f}")
else:
    print("No SEAHORSE metrics found in the file.")

print("-" * 50)

# Print some basic statistics
print("\nBasic Statistics:")
print(f"Number of summaries: {len(df_seahorse)}")
print(f"Average summary length: {df_seahorse['summary'].str.len().mean():.2f} characters")


SEAHORSE Metrics for GLIMPSE:
--------------------------------------------------
Comprehensible: 0.79 ± 0.19
Repetition: 0.93 ± 0.13
Grammar: 0.51 ± 0.24
Attribution: 0.41 ± 0.10
Main ideas: 0.15 ± 0.09
Conciseness: 0.18 ± 0.07
--------------------------------------------------

Basic Statistics:
Number of summaries: 6658
Average summary length: 220.27 characters


B. Extractive candidates

In [ ]:
import os

# Define the base paths for PubMed processed data and output
processed_dir = "/content/drive/MyDrive/glimpse/pubmed/proccessed"
output_dir = "/content/drive/MyDrive/glimpse/pubmed/extractive_summaries"

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Path to the PubMed test dataset
dataset_path = f"{processed_dir}/processed_test.csv"

# Check if input file exists
if not os.path.exists(dataset_path):
    print(f"Error: Input file {dataset_path} does not exist.")
else:
    print(f"Processing PubMed extractive summarization...")

    # Run the extractive summarization script
    exit_code = os.system(f"""
    python /content/drive/MyDrive/glimpse/glimpse-mds/glimpse/data_loading/generate_extractive_candidates.py \
        --dataset_path {dataset_path} \
        --output_dir {output_dir}
    """)

    if exit_code != 0:
        print(f"❌ Error: Script failed with exit code {exit_code}")
    else:
        print(f"✅ Extractive summaries for PubMed generated successfully!")

print("🎯 Task completed!")


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/glimpse/pubmed/extractive_summaries/extractive_sentences.csv")

In [ ]:
print(df.shape)
print(df.head())
print(df.columns)

In [ ]:
#Step 1: (Optional) Add id Column (if not present)
import pandas as pd

# Load your extractive summary file
df = pd.read_csv("/content/drive/MyDrive/glimpse/pubmed/extractive_summaries/extractive_sentences-_-processed_test-_-none-_-2025-03-27-11-58-15.csv")

# Add 'id' column
df["id"] = df["index"]

# Save it
df.to_csv("/content/drive/MyDrive/glimpse/pubmed/extractive_summaries/extractive_sentences.csv", index=False)


In [ ]:
import pandas as pd

# Load your full extractive summaries dataset
input_path = "/content/drive/MyDrive/glimpse/pubmed/extractive_summaries/extractive_sentences.csv"
df = pd.read_csv(input_path)

print(f"Original dataset shape: {df.shape}")

# Sample 1000 unique IDs
sampled_ids = df['id'].drop_duplicates().sample(n=1000, random_state=42)

# Select only rows corresponding to those sampled IDs
df_small = df[df['id'].isin(sampled_ids)].reset_index(drop=True)

print(f"New sampled dataset shape: {df_small.shape}")

# Save the smaller dataset
output_path = "/content/drive/MyDrive/glimpse/pubmed/abstractive_summaries/sample_1000_ids.csv"
df_small.to_csv(output_path, index=False)
print(f"✅ Smaller dataset saved at {output_path}")

In [ ]:
#Step 2: Run RSA Reranking

!python /content/drive/MyDrive/glimpse/glimpse-mds/glimpse/src/compute_rsa.py \
  --model_name facebook/bart-large-cnn \
  --summaries /content/drive/MyDrive/glimpse/pubmed/abstractive_summaries/sample_1000_ids.csv \
  --output_dir /content/drive/MyDrive/glimpse/pubmed/extractive_summaries/ \
  --device cuda

In [10]:
# generating glimpse_speaker and glimpse_unique with cleaning

import pickle
import pandas as pd
import numpy as np
from pathlib import Path

# Paths
pk_dir_path = "/content/drive/MyDrive/glimpse/pubmed/extractive_summaries"
output_speaker = "/content/drive/MyDrive/glimpse/pubmed/extractive_summaries/pubmed_extractive_speaker.csv"
output_unique = "/content/drive/MyDrive/glimpse/pubmed/extractive_summaries/pubmed_extractive_unique.csv"

# ✅ Function to clean text
def clean_text(text):
    if isinstance(text, str):
        return text.replace("<s>", "").replace("</s>", "").replace("\n", " ").strip()
    return ""

# Lists to hold extracted data
rows_speaker = []
rows_unique = []

# Process each .pk file
for pk_file in Path(pk_dir_path).glob("*.pk"):
    print(f"Processing file: {pk_file}")
    try:
        # Load .pk file
        with open(pk_file, "rb") as f:
            data = pickle.load(f)

        # Extract "results" section
        results = data.get("results", [])

        for result in results:
            id_ = result["id"][0]  # Extract URL from tuple
            gold_summary = clean_text(result["gold"])  # ✅ Clean gold reference summary

            # ✅ Extract GLIMPSE-Speaker summaries (best_rsa)
            best_rsa = result.get("best_rsa", [])
            if isinstance(best_rsa, (np.ndarray, list)) and len(best_rsa) > 0:
                for summary in best_rsa:
                    cleaned_summary = clean_text(summary)
                    if len(cleaned_summary.split()) > 5:  # ✅ Remove very short summaries
                        rows_speaker.append({
                            "id": id_,
                            "Method": "GLIMPSE-Speaker",
                            "summary": cleaned_summary,
                            "gold": gold_summary
                        })

            # ✅ Extract GLIMPSE-Unique summaries (Top 3 based on consensuality score)
            initial_consensuality_scores = result.get("initial_consensuality_scores", None)
            if isinstance(initial_consensuality_scores, pd.Series) and not initial_consensuality_scores.empty:
                sorted_summaries = initial_consensuality_scores.sort_values(ascending=False).index[:3]
                for summary in sorted_summaries:
                    cleaned_summary = clean_text(summary)
                    if len(cleaned_summary.split()) > 5:  # ✅ Remove very short summaries
                        rows_unique.append({
                            "id": id_,
                            "Method": "GLIMPSE-Unique",
                            "summary": cleaned_summary,
                            "gold": gold_summary
                        })

    except Exception as e:
        print(f"⚠️ Error processing {pk_file}: {e}")

# Convert lists to DataFrames
df_speaker = pd.DataFrame(rows_speaker)
df_unique = pd.DataFrame(rows_unique)

# Save extracted and cleaned data to CSV files
df_speaker.to_csv(output_speaker, index=False)
df_unique.to_csv(output_unique, index=False)

print("✅ GLIMPSE-Speaker summaries saved to:", output_speaker)
print("✅ GLIMPSE-Unique summaries saved to:", output_unique)


Processing file: /content/drive/MyDrive/glimpse/pubmed/extractive_summaries/sample_1000_ids-_-r3-_-rsa_reranked-facebook-bart-large-cnn.pk
✅ GLIMPSE-Speaker summaries saved to: /content/drive/MyDrive/glimpse/pubmed/extractive_summaries/pubmed_extractive_speaker.csv
✅ GLIMPSE-Unique summaries saved to: /content/drive/MyDrive/glimpse/pubmed/extractive_summaries/pubmed_extractive_unique.csv


In [ ]:
# Step 3: Evaluate with ROUGE
#!python /content/drive/MyDrive/glimpse/glimpse-mds/glimpse/evaluate/evaluate_common_metrics_samples.py --summaries /content/drive/MyDrive/glimpse/pubmed/extractive_summaries/rough_pubmed_extractive_speaker.csv
!python /content/drive/MyDrive/glimpse/glimpse-mds/glimpse/evaluate/evaluate_common_metrics_samples.py --summaries /content/drive/MyDrive/glimpse/pubmed/extractive_summaries/rough_pubmed_extractive_unique.csv


In [12]:
# Paths to the updated CSVs after running ROUGE evaluation
output_speaker = "/content/drive/MyDrive/glimpse/pubmed/extractive_summaries/rough_pubmed_extractive_speaker.csv"
output_unique = "/content/drive/MyDrive/glimpse/pubmed/extractive_summaries/rough_pubmed_extractive_unique.csv"

# Load CSV files
df_speaker = pd.read_csv(output_speaker)
df_unique = pd.read_csv(output_unique)

# Check if ROUGE scores exist
print("Columns in GLIMPSE-Speaker CSV:", df_speaker.columns)
print("Columns in GLIMPSE-Unique CSV:", df_unique.columns)

Columns in GLIMPSE-Speaker CSV: Index(['id', 'Method', 'summary', 'gold', 'common/rouge1', 'common/rouge2',
       'common/rougeL', 'common/rougeLsum'],
      dtype='object')
Columns in GLIMPSE-Unique CSV: Index(['id', 'Method', 'summary', 'gold', 'common/rouge1', 'common/rouge2',
       'common/rougeL', 'common/rougeLsum'],
      dtype='object')


In [13]:
# Select only ROUGE score columns
rouge_columns = ["common/rouge1", "common/rouge2", "common/rougeL", "common/rougeLsum"]

# Compute mean and std ROUGE scores (as decimals, not percentages)
mean_speaker = df_speaker[rouge_columns].mean()  # Removed * 100
std_speaker = df_speaker[rouge_columns].std()    # Removed * 100
mean_unique = df_unique[rouge_columns].mean()    # Removed * 100
std_unique = df_unique[rouge_columns].std()      # Removed * 100

# Create a formatted table
print("\nROUGE Scores:")
print("=" * 60)
print(f"{'Metric':<12} {'GLIMPSE-Speaker':>22} {'GLIMPSE-Unique':>22}")
print("-" * 60)

metrics_display = {
    "common/rouge1": "ROUGE-1",
    "common/rouge2": "ROUGE-2",
    "common/rougeL": "ROUGE-L",
    "common/rougeLsum": "ROUGE-Lsum"
}

for col in rouge_columns:
    metric_name = metrics_display[col]
    # Format scores to match paper style (0.XX ±0.XX)
    speaker_score = f"{mean_speaker[col]:.2f} ±{std_speaker[col]:.2f}"
    unique_score = f"{mean_unique[col]:.2f} ±{std_unique[col]:.2f}"
    print(f"{metric_name:<12} {speaker_score:>22} {unique_score:>22}")

print("=" * 60)


ROUGE Scores:
Metric              GLIMPSE-Speaker         GLIMPSE-Unique
------------------------------------------------------------
ROUGE-1                  0.14 ±0.08             0.13 ±0.07
ROUGE-2                  0.05 ±0.06             0.04 ±0.05
ROUGE-L                  0.10 ±0.06             0.09 ±0.05
ROUGE-Lsum               0.10 ±0.06             0.09 ±0.05


In [14]:
#preparing df s to for seahorse calculation

import pandas as pd

# Load CSV
file_path = "/content/drive/MyDrive/glimpse/pubmed/extractive_summaries/seahorse_pubmed_extractive_speaker.csv"
#file_path = "/content/drive/MyDrive/glimpse/pubmed/extractive_summaries/seahorse_pubmed_extractive_unique.csv"
df = pd.read_csv(file_path)

# Rename 'gold' → 'text'
df.rename(columns={"gold": "text"}, inplace=True)

# Save the updated file
df.to_csv(file_path, index=False)

print("✅ CSV column renamed successfully!")

✅ CSV column renamed successfully!


In [ ]:
# Step 4: Evaluate with SEAHORSE (Human-like metrics)
import pandas as pd

# Path to the CSV file
#file_path = "/content/drive/MyDrive/glimpse/pubmed/extractive_summaries/seahorse_pubmed_extractive_speaker.csv"
file_path = "/content/drive/MyDrive/glimpse/pubmed/extractive_summaries/seahorse_pubmed_extractive_unique.csv"

for question in range(1, 7):
    print(f"⚡ Running Seahorse for Question {question}...")

    # Reload dataset before each run to avoid missing columns
    df = pd.read_csv(file_path)

    if "text" not in df.columns or "summary" not in df.columns:
        print("❌ ERROR: 'text' and 'summary' columns missing before running Seahorse. Stopping execution.")
        break  # Stop if important columns are missing

    # Run the Seahorse evaluation script
    !python /content/drive/MyDrive/glimpse/glimpse-mds/glimpse/evaluate/Copyof_evaluate_seahorse_metrics_samples.py \
        --summaries {file_path} \
        --question {question} \
        --batch_size 8 \
        --device cuda

    # ✅ Verify columns after running Seahorse
    df = pd.read_csv(file_path)
    print(f"✅ After Question {question}, Columns:", df.columns.tolist())

    if "text" not in df.columns or "summary" not in df.columns:
        print("❌ ERROR: 'text' and 'summary' columns missing after running Seahorse. Stopping execution.")
        break


In [1]:
import pandas as pd

# Read the speaker summaries file
#df_seahorse = pd.read_csv("/content/drive/MyDrive/glimpse/pubmed/extractive_summaries/seahorse_pubmed_extractive_speaker.csv")
df_seahorse = pd.read_csv("/content/drive/MyDrive/glimpse/pubmed/extractive_summaries/seahorse_pubmed_extractive_unique.csv")

# Get SEAHORSE metrics (only the proba_1 columns)
seahorse_cols = [col for col in df_seahorse.columns if col.startswith("SHMetric") and col.endswith("proba_1")]

print("\nSEAHORSE Metrics for GLIMPSE:")
print("-" * 50)

if seahorse_cols:
    for col in seahorse_cols:
        metric_name = col.split("/")[1]
        mean_score = df_seahorse[col].mean()
        std_score = df_seahorse[col].std()
        print(f"{metric_name}: {mean_score:.2f} ± {std_score:.2f}")
else:
    print("No SEAHORSE metrics found in the file.")

print("-" * 50)

# Print some basic statistics
print("\nBasic Statistics:")
print(f"Number of summaries: {len(df_seahorse)}")
print(f"Average summary length: {df_seahorse['summary'].str.len().mean():.2f} characters")


SEAHORSE Metrics for GLIMPSE:
--------------------------------------------------
Comprehensible: 0.81 ± 0.18
Repetition: 0.95 ± 0.10
Grammar: 0.53 ± 0.22
Attribution: 0.38 ± 0.07
Main ideas: 0.08 ± 0.08
Conciseness: 0.14 ± 0.05
--------------------------------------------------

Basic Statistics:
Number of summaries: 2969
Average summary length: 180.29 characters


## extension2: Model Comparison

In [2]:
import torch
print(f"Is CUDA available? {torch.cuda.is_available()}")
print(f"GPU Device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'No GPU detected'}")

Is CUDA available? False
GPU Device: No GPU detected


In [ ]:
import torch
import transformers
torch.backends.cudnn.benchmark = True  # Optimized GPU usage
transformers.utils.logging.set_verbosity_info()

In [ ]:
import os
import subprocess

# List of models to run
models = ["google/pegasus-large",
          #"t5-large"
           ]

# Define the output directory
output_dir = "/content/drive/MyDrive/glimpse/glimpse-mds/data/extension2"

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

for model in models:
    print(f"⚡ Running summarization with {model}...")

#/content/drive/MyDrive/glimpse/glimpse-mds/glimpse/data_loading/pegasus_generate_abstractive_candidates.py
#/content/drive/MyDrive/glimpse/glimpse-mds/glimpse/data_loading/T5_generate_abstractive_candidates.py
    command = f"""
    python /content/drive/MyDrive/glimpse/glimpse-mds/glimpse/data_loading/pegasus_generate_abstractive_candidates.py \
        --model_name {model} \
        --dataset_path /content/drive/MyDrive/glimpse/glimpse-mds/data/processed/all_reviews_2017.csv \
        --decoding_config beam_search \
        --batch_size 8 \
        --device cuda \
        --output_dir "{output_dir}"
    """

    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()

    print(stdout.decode())  # ✅ See if script is running
    print(stderr.decode())  # ✅ See errors if script is failing


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/glimpse/glimpse-mds/')
from rsasumm.rsa_reranker import RSAReranking
print("Import successful")

In [ ]:
#compute rsa score for t5 and pegasus summeries
# /content/drive/MyDrive/glimpse/glimpse-mds/data/extension2/t5-large/t5-large-_-all_reviews_2017-_-beam_search-_-trimmed-_-2025-03-27-08-36-37.csv
# /content/drive/MyDrive/glimpse/glimpse-mds/data/extension2/google_pegasus-large/google_pegasus-large-_-all_reviews_2017-_-beam_search-_-trimmed-_-2025-03-27-09-05-28.csv

!CUDA_LAUNCH_BLOCKING=1 python /content/drive/MyDrive/glimpse/glimpse-mds/glimpse/src/compute_rsa.py \
            --model_name facebook/bart-large-cnn \
            --summaries /content/drive/MyDrive/glimpse/glimpse-mds/data/extension2/google_pegasus-large/google_pegasus-large-_-all_reviews_2017-_-beam_search-_-trimmed-_-2025-03-27-09-05-28.csv \
            --output_dir /content/drive/MyDrive/glimpse/glimpse-mds/data/extension2/rsa_scores  \
            --device cuda

In [ ]:
# generating glimpse_speaker and glimpse_unique with cleaning

import pickle
import pandas as pd
import numpy as np
from pathlib import Path

# Paths
pk_dir_path = "/content/drive/MyDrive/glimpse/glimpse-mds/data/extension2/rsa_scores/pegasus"
output_speaker = "/content/drive/MyDrive/glimpse/glimpse-mds/data/extension2/rsa_scores/pegasus/pegasus_abstractive_speaker.csv"
output_unique = "/content/drive/MyDrive/glimpse/glimpse-mds/data/extension2/rsa_scores/pegasus/pegasus_abstractive_unique.csv"

# ✅ Function to clean text
def clean_text(text):
    if isinstance(text, str):
        return text.replace("<s>", "").replace("</s>", "").replace("\n", " ").strip()
    return ""

# Lists to hold extracted data
rows_speaker = []
rows_unique = []

# Process each .pk file
for pk_file in Path(pk_dir_path).glob("*.pk"):
    print(f"Processing file: {pk_file}")
    try:
        # Load .pk file
        with open(pk_file, "rb") as f:
            data = pickle.load(f)

        # Extract "results" section
        results = data.get("results", [])

        for result in results:
            id_ = result["id"][0]  # Extract URL from tuple
            gold_summary = clean_text(result["gold"])  # ✅ Clean gold reference summary

            # ✅ Extract GLIMPSE-Speaker summaries (best_rsa)
            best_rsa = result.get("best_rsa", [])
            if isinstance(best_rsa, (np.ndarray, list)) and len(best_rsa) > 0:
                for summary in best_rsa:
                    cleaned_summary = clean_text(summary)
                    if len(cleaned_summary.split()) > 5:  # ✅ Remove very short summaries
                        rows_speaker.append({
                            "id": id_,
                            "Method": "GLIMPSE-Speaker",
                            "summary": cleaned_summary,
                            "gold": gold_summary
                        })

            # ✅ Extract GLIMPSE-Unique summaries (Top 3 based on consensuality score)
            initial_consensuality_scores = result.get("initial_consensuality_scores", None)
            if isinstance(initial_consensuality_scores, pd.Series) and not initial_consensuality_scores.empty:
                sorted_summaries = initial_consensuality_scores.sort_values(ascending=False).index[:3]
                for summary in sorted_summaries:
                    cleaned_summary = clean_text(summary)
                    if len(cleaned_summary.split()) > 5:  # ✅ Remove very short summaries
                        rows_unique.append({
                            "id": id_,
                            "Method": "GLIMPSE-Unique",
                            "summary": cleaned_summary,
                            "gold": gold_summary
                        })

    except Exception as e:
        print(f"⚠️ Error processing {pk_file}: {e}")

# Convert lists to DataFrames
df_speaker = pd.DataFrame(rows_speaker)
df_unique = pd.DataFrame(rows_unique)

# Save extracted and cleaned data to CSV files
df_speaker.to_csv(output_speaker, index=False)
df_unique.to_csv(output_unique, index=False)

print("✅ GLIMPSE-Speaker summaries saved to:", output_speaker)
print("✅ GLIMPSE-Unique summaries saved to:", output_unique)


In [ ]:
#!python /content/drive/MyDrive/glimpse/glimpse-mds/glimpse/evaluate/evaluate_common_metrics_samples.py --summaries /content/drive/MyDrive/glimpse/glimpse-mds/data/extension2/rsa_scores/pegasus/rough_pegasus_abstractive_speaker.csv
!python /content/drive/MyDrive/glimpse/glimpse-mds/glimpse/evaluate/evaluate_common_metrics_samples.py --summaries /content/drive/MyDrive/glimpse/glimpse-mds/data/extension2/rsa_scores/pegasus/rough_pegasus_abstractive_unique.csv

In [3]:
# Paths to the updated CSVs after running ROUGE evaluation
output_speaker = "/content/drive/MyDrive/glimpse/glimpse-mds/data/extension2/rsa_scores/pegasus/rough_pegasus_abstractive_speaker.csv"
output_unique = "/content/drive/MyDrive/glimpse/glimpse-mds/data/extension2/rsa_scores/pegasus/rough_pegasus_abstractive_unique.csv"

# Load CSV files
df_speaker = pd.read_csv(output_speaker)
df_unique = pd.read_csv(output_unique)

# Check if ROUGE scores exist
print("Columns in GLIMPSE-Speaker CSV:", df_speaker.columns)
print("Columns in GLIMPSE-Unique CSV:", df_unique.columns)

Columns in GLIMPSE-Speaker CSV: Index(['id', 'Method', 'summary', 'gold', 'common/rouge1', 'common/rouge2',
       'common/rougeL', 'common/rougeLsum'],
      dtype='object')
Columns in GLIMPSE-Unique CSV: Index(['id', 'Method', 'summary', 'gold', 'common/rouge1', 'common/rouge2',
       'common/rougeL', 'common/rougeLsum'],
      dtype='object')


In [4]:
# Select only ROUGE score columns
rouge_columns = ["common/rouge1", "common/rouge2", "common/rougeL", "common/rougeLsum"]

# Compute mean ROUGE scores
mean_speaker = df_speaker[rouge_columns].mean()
mean_unique = df_unique[rouge_columns].mean()

# Print results
print("\n📌 Mean ROUGE Scores for GLIMPSE-Speaker:")
print(mean_speaker)

print("\n📌 Mean ROUGE Scores for GLIMPSE-Unique:")
print(mean_unique)


📌 Mean ROUGE Scores for GLIMPSE-Speaker:
common/rouge1       0.209226
common/rouge2       0.027800
common/rougeL       0.124781
common/rougeLsum    0.124781
dtype: float64

📌 Mean ROUGE Scores for GLIMPSE-Unique:
common/rouge1       0.208563
common/rouge2       0.027755
common/rougeL       0.124317
common/rougeLsum    0.124317
dtype: float64


In [5]:
# Select only ROUGE score columns
rouge_columns = ["common/rouge1", "common/rouge2", "common/rougeL", "common/rougeLsum"]

# Compute mean and std ROUGE scores (as decimals, not percentages)
mean_speaker = df_speaker[rouge_columns].mean()  # Removed * 100
std_speaker = df_speaker[rouge_columns].std()    # Removed * 100
mean_unique = df_unique[rouge_columns].mean()    # Removed * 100
std_unique = df_unique[rouge_columns].std()      # Removed * 100

# Create a formatted table
print("\nROUGE Scores:")
print("=" * 60)
print(f"{'Metric':<12} {'GLIMPSE-Speaker':>22} {'GLIMPSE-Unique':>22}")
print("-" * 60)

metrics_display = {
    "common/rouge1": "ROUGE-1",
    "common/rouge2": "ROUGE-2",
    "common/rougeL": "ROUGE-L",
    "common/rougeLsum": "ROUGE-Lsum"
}

for col in rouge_columns:
    metric_name = metrics_display[col]
    # Format scores to match paper style (0.XX ±0.XX)
    speaker_score = f"{mean_speaker[col]:.2f} ±{std_speaker[col]:.2f}"
    unique_score = f"{mean_unique[col]:.2f} ±{std_unique[col]:.2f}"
    print(f"{metric_name:<12} {speaker_score:>22} {unique_score:>22}")

print("=" * 60)


ROUGE Scores:
Metric              GLIMPSE-Speaker         GLIMPSE-Unique
------------------------------------------------------------
ROUGE-1                  0.21 ±0.08             0.21 ±0.08
ROUGE-2                  0.03 ±0.03             0.03 ±0.03
ROUGE-L                  0.12 ±0.04             0.12 ±0.04
ROUGE-Lsum               0.12 ±0.04             0.12 ±0.04


In [ ]:
#preparing df s to for seahorse calculation

import pandas as pd

# Load CSV
#file_path = "/content/drive/MyDrive/glimpse/glimpse-mds/data/extension2/rsa_scores/pegasus/seahorse_pegasus_abstractive_speaker.csv"
file_path = "/content/drive/MyDrive/glimpse/glimpse-mds/data/extension2/rsa_scores/pegasus/seahorse_pegasus_abstractive_unique.csv"
df = pd.read_csv(file_path)

# Rename 'gold' → 'text'
df.rename(columns={"gold": "text"}, inplace=True)

# Save the updated file
df.to_csv(file_path, index=False)

print("✅ CSV column renamed successfully!")

In [ ]:
#seahorse metrics calculation

import pandas as pd

# Path to the CSV file
#file_path = "/content/drive/MyDrive/glimpse/glimpse-mds/data/extension2/rsa_scores/pegasus/seahorse_pegasus_abstractive_speaker.csv"
file_path = "/content/drive/MyDrive/glimpse/glimpse-mds/data/extension2/rsa_scores/pegasus/seahorse_pegasus_abstractive_unique.csv"

for question in range(1, 7):
    print(f"⚡ Running Seahorse for Question {question}...")

    # Reload dataset before each run to avoid missing columns
    df = pd.read_csv(file_path)

    if "text" not in df.columns or "summary" not in df.columns:
        print("❌ ERROR: 'text' and 'summary' columns missing before running Seahorse. Stopping execution.")
        break  # Stop if important columns are missing

    # Run the Seahorse evaluation script
    !python /content/drive/MyDrive/glimpse/glimpse-mds/glimpse/evaluate/Copyof_evaluate_seahorse_metrics_samples.py \
        --summaries {file_path} \
        --question {question} \
        --batch_size 8 \
        --device cuda

    # ✅ Verify columns after running Seahorse
    df = pd.read_csv(file_path)
    print(f"✅ After Question {question}, Columns:", df.columns.tolist())

    if "text" not in df.columns or "summary" not in df.columns:
        print("❌ ERROR: 'text' and 'summary' columns missing after running Seahorse. Stopping execution.")
        break


In [6]:
import pandas as pd

# Read the speaker summaries file
#df_seahorse = pd.read_csv("/content/drive/MyDrive/glimpse/glimpse-mds/data/extension2/rsa_scores/pegasus/seahorse_pegasus_abstractive_speaker.csv")
df_seahorse = pd.read_csv("/content/drive/MyDrive/glimpse/glimpse-mds/data/extension2/rsa_scores/pegasus/seahorse_pegasus_abstractive_unique.csv")

# Get SEAHORSE metrics (only the proba_1 columns)
seahorse_cols = [col for col in df_seahorse.columns if col.startswith("SHMetric") and col.endswith("proba_1")]

print("\nSEAHORSE Metrics for GLIMPSE:")
print("-" * 50)

if seahorse_cols:
    for col in seahorse_cols:
        metric_name = col.split("/")[1]
        mean_score = df_seahorse[col].mean()
        std_score = df_seahorse[col].std()
        print(f"{metric_name}: {mean_score:.2f} ± {std_score:.2f}")
else:
    print("No SEAHORSE metrics found in the file.")

print("-" * 50)

# Print some basic statistics
print("\nBasic Statistics:")
print(f"Number of summaries: {len(df_seahorse)}")
print(f"Average summary length: {df_seahorse['summary'].str.len().mean():.2f} characters")


SEAHORSE Metrics for GLIMPSE:
--------------------------------------------------
Comprehensible: 0.64 ± 0.22
Repetition: 0.90 ± 0.12
Grammar: 0.45 ± 0.24
Attribution: 0.35 ± 0.02
Main ideas: 0.06 ± 0.06
Conciseness: 0.13 ± 0.03
--------------------------------------------------

Basic Statistics:
Number of summaries: 1455
Average summary length: 431.85 characters
